# Remapping MIDI instruments
In this notebook we study the MIDI instruments and merge similar instruments into one track.

In [1]:
import pretty_midi
import numpy as np
import collections
import os
import pypianoroll as ppr

In [2]:
# Get all filtered files
files = list()
dirName = os.path.join("..", "data_filtered") # path to filtered data
for (dirpath, dirnames, filenames) in os.walk(dirName):
    files += [os.path.join(dirpath, file) for file in filenames]
    
print("Number of files found: ", len(files))


Number of files found:  5051


In [3]:
# Show the instrument map
imap = pretty_midi.INSTRUMENT_MAP
pianos = imap[0:9] + imap[18:24]
print("1. Pianos:", pianos)
guitars = imap[24:32]
print("\n2. Guitars:", guitars)
basses = imap[32:40]
print("\n3. Basses:", basses)
strings = imap[40:56]
print("\n4. String:", strings)
lead = imap[56:88]
print("\n5. Lead:", lead)
others = imap[88:128]
print("\n6. Other:", others)

1. Pianos: ['Acoustic Grand Piano', 'Bright Acoustic Piano', 'Electric Grand Piano', 'Honky-tonk Piano', 'Electric Piano 1', 'Electric Piano 2', 'Harpsichord', 'Clavinet', 'Celesta', 'Rock Organ', 'Church Organ', 'Reed Organ', 'Accordion', 'Harmonica', 'Tango Accordion']

2. Guitars: ['Acoustic Guitar (nylon)', 'Acoustic Guitar (steel)', 'Electric Guitar (jazz)', 'Electric Guitar (clean)', 'Electric Guitar (muted)', 'Overdriven Guitar', 'Distortion Guitar', 'Guitar Harmonics']

3. Basses: ['Acoustic Bass', 'Electric Bass (finger)', 'Electric Bass (pick)', 'Fretless Bass', 'Slap Bass 1', 'Slap Bass 2', 'Synth Bass 1', 'Synth Bass 2']

4. String: ['Violin', 'Viola', 'Cello', 'Contrabass', 'Tremolo Strings', 'Pizzicato Strings', 'Orchestral Harp', 'Timpani', 'String Ensemble 1', 'String Ensemble 2', 'Synth Strings 1', 'Synth Strings 2', 'Choir Aahs', 'Voice Oohs', 'Synth Choir', 'Orchestra Hit']

5. Lead: ['Trumpet', 'Trombone', 'Tuba', 'Muted Trumpet', 'French Horn', 'Brass Section', 'Sy

In [32]:
# function for editing instruments
def edit_instruments(pm):
    new_inst = []
    for ins in pm.instruments:
        nam = ins.name.lower()
        key0 = "drum" in nam
        ins.name  = nam
        if(ins.is_drum or key0):
            ins.program = 0
            ins.is_drum = True
            new_inst.append(ins)
        else:        
            key1 = "bass" in nam
            key3 = "melody" in nam
            key4 = "lead" in nam
            key5 = "vocal" in nam
            key6 = "lyric" in nam
            is_rhythm = ins.program in range(0,9) or ins.program in range(18,32) or ...
            ins.program in range(40,47) or ins.program in range(48,55)
            is_bass   = ins.program in range(32,40) or key1
            is_lead   = ins.program in range(56,96) or key3 or key4 or key5 or key6
            if(is_lead):
                ins.program = 56 # trumpet
                new_inst.append(ins)
            elif(is_bass):
                ins.program = 34 # bass
                new_inst.append(ins)
            elif(is_rhythm):
                ins.program = 0# piano
                new_inst.append(ins)
    pm.instruments = new_inst
    return(pm)

# function for pianoroll of drums
def edit_drums_roll(pr):
    new_pr = np.zeros(pr.shape)
    new_pr[:,35] = np.sum(pr[:,(35,36)], axis=1)             # kick
    new_pr[:,38] = np.sum(pr[:,(37,38,39,40,91,93)], axis=1) # snare
    new_pr[:,42] = np.sum(pr[:,(42,70,82)], axis=1)          # hat closed
    new_pr[:,44] = np.sum(pr[:,(44,46)], axis=1)             # hat open
    new_pr[:,49] = np.sum(pr[:,(49,52,55,57)], axis=1)       # crash
    new_pr[:,51] = np.sum(pr[:,(51,53,56,59)], axis=1)       # ride
    new_pr[:,43] = np.sum(pr[:,(41,43,45)], axis=1)          # low tom
    new_pr[:,47] = np.sum(pr[:,(47,48)], axis=1)             # middle tom
    new_pr[:,50] = np.sum(pr[:,(50,60,61)], axis=1)          # high tom
    return(new_pr)
    
# function for editing drums
def edit_drums(roll):
    idx  = 0
    for trk in roll.tracks:
        idx +=1
        if(trk.is_drum):
            roll.tracks[idx-1].pianoroll = edit_drums_roll(trk.pianoroll)
    return(roll)

# get indices of merged tracks
def get_merged_idx(roll, iref):
    irem = []
    j = 0
    for trk in roll.tracks:
        j += 1
        if(trk.is_drum):
            num = -1
        else:
            num = trk.program
        if(num==iref):
            irem.append(j-1)
    return irem

# downsample and merge some tracks
def process(fn):
    pm  = pretty_midi.PrettyMIDI(fn)
    pm  = edit_instruments(pm)
    pm.write("./temp.mid")
    roll = ppr.parse("./temp.mid", beat_resolution = 4)
    i0 = get_merged_idx(roll, 0)
    if(len(i0)> 1):
        roll.merge_tracks(i0, mode = 'any', remove_merged = True, is_drum = False, name = "piano", program = 0)
    i1 = get_merged_idx(roll, 34)
    if(len(i1)> 1):
        roll.merge_tracks(i1, mode = 'any', remove_merged = True, is_drum = False, name = "bass", program = 34)
    i2 = get_merged_idx(roll, 56)
    if(len(i2)> 1):
        roll.merge_tracks(i2, mode = 'any', remove_merged = True, is_drum = False, name = "melody", program = 56)   
    i3 = get_merged_idx(roll, -1)
    if(len(i3)> 1):
        roll.merge_tracks(i3, mode = 'any', remove_merged = True, is_drum = True, name = "drum", program = 0)
    roll = edit_drums(roll)
    postfix = fn.split(os.sep)[-1]
    out = os.path.join('..','cm4',postfix)
    roll.binarize()
    roll.write(out)

In [34]:
# Run merging and downsampling for all songs
idx = 0
for file in files:
    idx += 1
    try:
        process(file)
    except:
        print(idx, "<-- failed!", )
        pass
    if(idx % 500 == 0):
        print(idx)

130 <-- failed!
500
1000
1500
1671 <-- failed!
1837 <-- failed!
2000
2345 <-- failed!
2500
3000
3437 <-- failed!
3500
3867 <-- failed!
3947 <-- failed!
4000
4103 <-- failed!
4500
4981 <-- failed!
5000
